In [ ]:
# !wget https://raw.githubusercontent.com/alexmascension/revisit_reynolds_fb/master/requirements.txt
# !pip install -r requirements.txt

In [ ]:
import scanpy as sc
import scanpy.external as sce

import pandas as pd
import numpy as np
import os
from functools import reduce
import gseapy as gp

import triku as tk

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import scipy.stats as sts

import gc

In [ ]:
# To print versions of imports 

import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    try:
        name, version = module.split('==')
        if name in changed_imported_modules:
            print(name + '\t' + version)
    except:
        pass

In [ ]:
seed = 0

**Set this variable to True if the notebook crashes because of RAM failure. Also, don't run the notebook from beginning to end, and run 
the cells were processed adatas are loaded.**

In [ ]:
del_intermediates = False

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 70  # Set this to make higher quality figures

In [ ]:
def assign_cats(adata, dict_cats, column_groupby='leiden', quantile_gene_sel=0.7, do_return=False, intermediate_states=False, diff=0.05, 
                key_added='assigned_cats', min_score=0.6, others_name='Normal'):
    """
    This functions uses a set of genes assigned to different categories so that leiden clusters can be assigned to one of these categories.
    For example, to categorize fibroblasts from pericytes, endothelial cells, or cells with high mitochondrial content.
    It could be done with each cell individually, but it is better to use clusters to discern the different categories because
    the method, although efficient, can sometimes be noisy due to the noisiness of the sc datasets.
    """
    
    for cat in list(dict_cats.keys()):
        mat_cat = np.zeros((len(adata), len(dict_cats[cat])), dtype=float)
        
        for gene_idx, gene in enumerate(dict_cats[cat]):
            try:
                mat_cat[:, gene_idx] = np.asarray(np.dot(adata.obsp['connectivities'], adata[:, gene].X).todense()).ravel() / adata.uns['neighbors']['params']['n_neighbors']
                mat_cat[mat_cat[:, gene_idx] > 0, gene_idx] = np.argsort(np.argsort(mat_cat[mat_cat[:, gene_idx] > 0, gene_idx]))
                mat_cat[:, gene_idx] /= np.max(mat_cat[:, gene_idx])
            except:
                print(f'Gene {gene} is not on the list')    
            
        sum_mat_cat = np.asarray(mat_cat.mean(1)).ravel()       
        adata.obs[cat] = sum_mat_cat
    
    score_per_cluster = adata.obs[[column_groupby] + list(dict_cats.keys())].groupby(column_groupby).quantile(quantile_gene_sel)
    max_cat_dict_std = dict(zip(score_per_cluster.std(1).index, score_per_cluster.std(1).values))
    adata.obs[f'{key_added}_std'] = [max_cat_dict_std[i] for i in adata.obs[column_groupby]]
    max_cat_dict_mean = dict(zip(score_per_cluster.mean(1).index, score_per_cluster.mean(1).values))
    adata.obs[f'{key_added}_mean'] = [max_cat_dict_mean[i] for i in adata.obs[column_groupby]]
    max_cat_dict_max = dict(zip(score_per_cluster.max(1).index, score_per_cluster.max(1).values))
    adata.obs[f'{key_added}_max'] = [max_cat_dict_max[i] for i in adata.obs[column_groupby]]
    adata.obs[f'{key_added}_CV'] = adata.obs[f'{key_added}_mean'] / adata.obs[f'{key_added}_std']
    
    for cat in score_per_cluster.columns:
        max_cat_dict = dict(zip(score_per_cluster.index, score_per_cluster[cat].values))        
        adata.obs[f'{key_added}_{cat}'] = [max_cat_dict[i] for i in adata.obs[column_groupby]]
    
    if intermediate_states: # For each cluster we will identify which categories are close to the highest one, and merge their names.
        list_names_cats_per_cluster = []
        for cluster in score_per_cluster.index:
            scores_cluster = score_per_cluster.loc[cluster]
            scores_cluster = scores_cluster[scores_cluster > scores_cluster.max() - diff]
            list_names_cats_per_cluster.append('/'.join(scores_cluster.index))
        
        final_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, list_names_cats_per_cluster))
    else:        
        final_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, score_per_cluster.idxmax(axis=1).values))
    
    adata.obs[f'{key_added}'] = [str(final_cat_dict[i]) for i in adata.obs[column_groupby]]
    
    adata.obs[f'{key_added}'][adata.obs[f'{key_added}_max'] < min_score] = others_name
    
    if do_return:
        return score_per_cluster

# Reynolds et al. 2020 dataset download and preprocess
In this section we are going to download and process the anndata files for the 5 healthy samples (S1 to S5). We are first going to use the processed files from Reynolds et al., because they contain the main fb populations, which we are interested in (FB1-3). With that, we can partially replicate the analysis. However, the data richness is not that good (the UMAPs are more *blooby*) so we preprocess the FASTQ files on our own, and used the processed adatas.

Once we have our own anndatas, the preprocessing is the following:
* Assign FB types based on Reynolds adatas to our adatas. Some cells will be unassigned.
* QC metrics 
* Filter genes (min counts = 30)
* Set raw X
* Normalize and log1p
* PCA + triku + neighbors + umap + leiden
* Use  `assign_cats` to assign leiden clusters to cell populations with selected markers (markers for each dataset may vary!)
* Filter adata to retain only fibroblasts
* Filter genes to remove 0 counts
* PCA + triku + neighbors + umap + leiden [leiden is not used here but may be used later]
* Check if strange populations appear and filter them in `assign_cats`, then repeat the last steps.

In [ ]:
os.getcwd()

In [ ]:
reynolds_dir = 'reynolds_2020'
os.makedirs(reynolds_dir, exist_ok=True)

papers_dir = 'papers_genes_bad_quality'
os.makedirs(papers_dir, exist_ok=True)

In [ ]:
try:
    stress_genes = np.loadtxt('papers_genes_bad_quality/stress_genes.txt', dtype=str)
    hypoxia_genes = np.loadtxt('papers_genes_bad_quality/hypoxia_genes.txt', dtype=str)
except:
    stress_genes = np.loadtxt('https://raw.githubusercontent.com/alexmascension/revisit_reynolds_fb/master/papers_genes_bad_quality/stress_genes.txt', dtype=str)
    hypoxia_genes = np.loadtxt('https://raw.githubusercontent.com/alexmascension/revisit_reynolds_fb/master/papers_genes_bad_quality/hypoxia_genes.txt', dtype=str)

In [ ]:
adata_reynolds = sc.read('submission_210120.h5ad', backup_url='https://zenodo.org/record/4536165/files/submission_210120.h5ad')
adata_reynolds_healthy = adata_reynolds[adata_reynolds.obs['Status'] == 'Healthy']
adata_reynolds_disease = adata_reynolds[adata_reynolds.obs['Status'].isin(['Eczema', 'Psoriasis'])]

# Analysis on healthy populations 

In [ ]:
for adata_name in ['adata_reynolds_healthy']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.normalize_per_cell(eval(adata_name))
    sc.pp.log1p(eval(adata_name))
    sc.pp.highly_variable_genes(eval(adata_name))
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_healthy, color=['full_clustering'])

In [ ]:
adata_reynolds_healthy_fb = adata_reynolds_healthy[adata_reynolds_healthy.obs['full_clustering'].isin(['F1', 'F2', 'F3'])]
adata_reynolds_healthy_ve = adata_reynolds_healthy[adata_reynolds_healthy.obs['full_clustering'].isin(['VE1', 'VE2', 'VE3'])]
adata_reynolds_healthy_per = adata_reynolds_healthy[adata_reynolds_healthy.obs['full_clustering'].isin(['Pericyte_1', 'Pericyte_2'])]
adata_reynolds_healthy_krt = adata_reynolds_healthy[adata_reynolds_healthy.obs['full_clustering'].isin(['Differentiated_KC', 'Differentiated_KC*', 'Undifferentiated_KC'])]
adata_reynolds_healthy_lymphoid = adata_reynolds_healthy[adata_reynolds_healthy.obs['full_clustering'].isin(['Tc', 'Tc17_Th17', 'Tc_IL13_IL22', 'Th', 'Treg', 'ILC1_3', 'ILC1_NK', 'ILC2'])]
adata_reynolds_healthy_APC = adata_reynolds_healthy[adata_reynolds_healthy.obs['full_clustering'].isin(['Macro_1', 'Macro_2', 'Inf_mac', 'DC1', 'DC2', 'LC_1', 'LC_2', 'LC_3', 'Mono_mac', 'MigDC', 'MoDC_1', 'moDC_2', 'mo_DC3'])]

In [ ]:
samples, counts = np.unique(adata_reynolds_healthy_fb.obs['sample_id'], return_counts=True)
adata_reynolds_healthy_fb = adata_reynolds_healthy_fb[adata_reynolds_healthy_fb.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_healthy_ve.obs['sample_id'], return_counts=True)
adata_reynolds_healthy_ve = adata_reynolds_healthy_ve[adata_reynolds_healthy_ve.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_healthy_per.obs['sample_id'], return_counts=True)
adata_reynolds_healthy_per = adata_reynolds_healthy_per[adata_reynolds_healthy_per.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_healthy_krt.obs['sample_id'], return_counts=True)
adata_reynolds_healthy_krt = adata_reynolds_healthy_krt[adata_reynolds_healthy_krt.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_healthy_lymphoid.obs['sample_id'], return_counts=True)
adata_reynolds_healthy_lymphoid = adata_reynolds_healthy_lymphoid[adata_reynolds_healthy_lymphoid.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_healthy_APC.obs['sample_id'], return_counts=True)
adata_reynolds_healthy_APC = adata_reynolds_healthy_APC[adata_reynolds_healthy_APC.obs['sample_id'].isin(samples[counts > 10])]

In [ ]:
for adata_name in ['adata_reynolds_healthy_fb', 'adata_reynolds_healthy_ve', 'adata_reynolds_healthy_per', 
                   'adata_reynolds_healthy_krt', 'adata_reynolds_healthy_lymphoid', 'adata_reynolds_healthy_APC']:
    sc.pp.filter_genes(eval(adata_name), min_counts=1)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)
    sc.tl.leiden(eval(adata_name), resolution=2.5, random_state=seed)

In [ ]:
adata_reynolds_healthy_fb.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_fb.h5ad')
adata_reynolds_healthy_ve.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_ve.h5ad')
adata_reynolds_healthy_per.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_per.h5ad')
adata_reynolds_healthy_krt.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_krt.h5ad')
adata_reynolds_healthy_lymphoid.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_lymphoid.h5ad')
adata_reynolds_healthy_APC.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_APC.h5ad')

**RUN THIS CELL IF YOU ARE LOW ON RAM!**

In [ ]:
adata_reynolds_healthy_fb = sc.read(reynolds_dir + '/adata_reynolds_healthy_fb.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_healthy_fb.h5ad?download=1')
adata_reynolds_healthy_ve = sc.read(reynolds_dir + '/adata_reynolds_healthy_ve.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_healthy_ve.h5ad?download=1')
adata_reynolds_healthy_per = sc.read(reynolds_dir + '/adata_reynolds_healthy_per.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_healthy_per.h5ad?download=1')
adata_reynolds_healthy_krt = sc.read(reynolds_dir + '/adata_reynolds_healthy_krt.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_healthy_krt.h5ad?download=1')
adata_reynolds_healthy_lymphoid = sc.read(reynolds_dir + '/adata_reynolds_healthy_lymphoid.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_healthy_lymphoid.h5ad?download=1')
adata_reynolds_healthy_APC = sc.read(reynolds_dir + '/adata_reynolds_healthy_APC.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_healthy_APC.h5ad?download=1')

In [ ]:
dict_cats = {'Stress': stress_genes, 'Hypoxia': hypoxia_genes}

### Fibro

In [ ]:
assign_cats(adata_reynolds_healthy_fb, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=True, min_score=0.5,
            key_added='hypoxia_stress', )
sc.pl.umap(adata_reynolds_healthy_fb, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_healthy_fb, color=[i for i in stress_genes if i in adata_reynolds_healthy_fb.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_healthy_fb, color=[i for i in hypoxia_genes if i in adata_reynolds_healthy_fb.var_names], cmap=magma, use_raw=False)

### VE

In [ ]:
assign_cats(adata_reynolds_healthy_ve, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=True, min_score=0.5,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_healthy_ve, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_healthy_ve, color=[i for i in stress_genes if i in adata_reynolds_healthy_ve.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_healthy_ve, color=[i for i in hypoxia_genes if i in adata_reynolds_healthy_ve.var_names], cmap=magma, use_raw=False)

### PER

In [ ]:
assign_cats(adata_reynolds_healthy_per, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=True, min_score=0.5,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_healthy_per, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_healthy_per, color=[i for i in hypoxia_genes if i in adata_reynolds_healthy_per.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_healthy_per, color=[i for i in stress_genes if i in adata_reynolds_healthy_per.var_names], cmap=magma, use_raw=False)

### KRT

In [ ]:
assign_cats(adata_reynolds_healthy_krt, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=True, min_score=0.47,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_healthy_krt, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_healthy_krt, color=[i for i in hypoxia_genes if i in adata_reynolds_healthy_krt.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_healthy_krt, color=[i for i in stress_genes if i in adata_reynolds_healthy_krt.var_names], cmap=magma, use_raw=False)

### LYMPHOID

In [ ]:
assign_cats(adata_reynolds_healthy_lymphoid, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=True, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_healthy_lymphoid, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_healthy_lymphoid, color=[i for i in hypoxia_genes if i in adata_reynolds_healthy_lymphoid.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_healthy_lymphoid, color=[i for i in stress_genes if i in adata_reynolds_healthy_lymphoid.var_names], cmap=magma, use_raw=False)

### APC

In [ ]:
assign_cats(adata_reynolds_healthy_APC, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress', )
sc.pl.umap(adata_reynolds_healthy_APC, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_healthy_APC, color=[i for i in hypoxia_genes if i in adata_reynolds_healthy_APC.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_healthy_APC, color=[i for i in stress_genes if i in adata_reynolds_healthy_APC.var_names], cmap=magma, use_raw=False)

In [ ]:
adata_reynolds_healthy_fb.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_fb.h5ad')
adata_reynolds_healthy_ve.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_ve.h5ad')
adata_reynolds_healthy_per.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_per.h5ad')
adata_reynolds_healthy_krt.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_krt.h5ad')
adata_reynolds_healthy_lymphoid.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_lymphoid.h5ad')
adata_reynolds_healthy_APC.write_h5ad(reynolds_dir + '/adata_reynolds_healthy_APC.h5ad')

# Preparing Figure 1B

In [ ]:
from matplotlib.patches import Patch 

dict_colors = {'Hypoxia': '#e14b67', 'Stress': '#d98c58', 'Stress/Hypoxia': '#DD6C60', 'Normal': '#009f61'}

for adata_name in ['adata_reynolds_healthy_fb', 'adata_reynolds_healthy_ve', 'adata_reynolds_healthy_per', 
                   'adata_reynolds_healthy_krt', 'adata_reynolds_healthy_lymphoid', 'adata_reynolds_healthy_APC']:
    eval(adata_name).uns['hypoxia_stress_colors'] = [dict_colors[i] for i in sorted(set(eval(adata_name).obs['hypoxia_stress'].values))]
    
fig, axs = plt.subplots(2, 3)

titles = ['Fibroblasts', 'Vascular endothelium', 'Pericytes', 'Keratinocytes', 'Lymphoid', 'APCs']

for idx, adata_name in enumerate(['adata_reynolds_healthy_fb', 'adata_reynolds_healthy_ve', 'adata_reynolds_healthy_per', 
                   'adata_reynolds_healthy_krt', 'adata_reynolds_healthy_lymphoid', 'adata_reynolds_healthy_APC']):
    ax = axs.ravel()[idx]
    title = titles[idx]
    
    # We are going to reduce the number of cells to make the pdf load easily
    n_sub = min(len(eval(adata_name)), 4000)
    adata_sub = sc.pp.subsample(eval(adata_name), n_obs=n_sub, random_state=seed, copy=True)
    
    sc.pl.umap(adata_sub, color=['hypoxia_stress'], title=title, frameon=False, ax=ax, legend_loc=None, show=False, s=8)

legend = plt.legend(handles = [Patch(color=dict_colors[i], label=i) for i in dict_colors.keys()], 
                    bbox_to_anchor=(1.05, 1.5), frameon=False)

os.makedirs('figures', exist_ok=True)
plt.savefig('figures/umapF1B.pdf', bbox_inches='tight')

In [ ]:
if del_intermediates:
    del [adata_reynolds_healthy_fb, adata_reynolds_healthy_ve, adata_reynolds_healthy_per, adata_reynolds_healthy_krt, 
         adata_reynolds_healthy_lymphoid, adata_reynolds_healthy_APC]
    gc.collect()

# Analysis on diseased populations 

In [ ]:
for adata_name in ['adata_reynolds_disease']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.normalize_per_cell(eval(adata_name))
    sc.pp.log1p(eval(adata_name))
    sc.pp.highly_variable_genes(eval(adata_name))
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_disease, color=['full_clustering'])

In [ ]:
adata_reynolds_disease_fb = adata_reynolds_disease[adata_reynolds_disease.obs['full_clustering'].isin(['F1', 'F2', 'F3'])]
adata_reynolds_disease_ve = adata_reynolds_disease[adata_reynolds_disease.obs['full_clustering'].isin(['VE1', 'VE2', 'VE3'])]
adata_reynolds_disease_per = adata_reynolds_disease[adata_reynolds_disease.obs['full_clustering'].isin(['Pericyte_1', 'Pericyte_2'])]
adata_reynolds_disease_krt = adata_reynolds_disease[adata_reynolds_disease.obs['full_clustering'].isin(['Differentiated_KC', 'Differentiated_KC*', 'Undifferentiated_KC'])]
adata_reynolds_disease_lymphoid = adata_reynolds_disease[adata_reynolds_disease.obs['full_clustering'].isin(['Tc', 'Tc17_Th17', 'Tc_IL13_IL22', 'Th', 'Treg', 'ILC1_3', 'ILC1_NK', 'ILC2'])]
adata_reynolds_disease_APC = adata_reynolds_disease[adata_reynolds_disease.obs['full_clustering'].isin(['Macro_1', 'Macro_2', 'Inf_mac', 'DC1', 'DC2', 'LC_1', 'LC_2', 'LC_3', 'Mono_mac', 'MigDC', 'MoDC_1', 'moDC_2', 'mo_DC3'])]

In [ ]:
samples, counts = np.unique(adata_reynolds_disease_fb.obs['sample_id'], return_counts=True)
adata_reynolds_disease_fb = adata_reynolds_disease_fb[adata_reynolds_disease_fb.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_disease_ve.obs['sample_id'], return_counts=True)
adata_reynolds_disease_ve = adata_reynolds_disease_ve[adata_reynolds_disease_ve.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_disease_per.obs['sample_id'], return_counts=True)
adata_reynolds_disease_per = adata_reynolds_disease_per[adata_reynolds_disease_per.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_disease_krt.obs['sample_id'], return_counts=True)
adata_reynolds_disease_krt = adata_reynolds_disease_krt[adata_reynolds_disease_krt.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_disease_lymphoid.obs['sample_id'], return_counts=True)
adata_reynolds_disease_lymphoid = adata_reynolds_disease_lymphoid[adata_reynolds_disease_lymphoid.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_disease_APC.obs['sample_id'], return_counts=True)
adata_reynolds_disease_APC = adata_reynolds_disease_APC[adata_reynolds_disease_APC.obs['sample_id'].isin(samples[counts > 10])]

In [ ]:
for adata_name in ['adata_reynolds_disease_fb', 'adata_reynolds_disease_ve', 'adata_reynolds_disease_per', 
                   'adata_reynolds_disease_krt', 'adata_reynolds_disease_lymphoid', 'adata_reynolds_disease_APC']:
    sc.pp.filter_genes(eval(adata_name), min_counts=1)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)
    sc.tl.leiden(eval(adata_name), resolution=2.5, random_state=seed)

In [ ]:
adata_reynolds_disease_fb.write_h5ad(reynolds_dir + '/adata_reynolds_disease_fb.h5ad')
adata_reynolds_disease_ve.write_h5ad(reynolds_dir + '/adata_reynolds_disease_ve.h5ad')
adata_reynolds_disease_per.write_h5ad(reynolds_dir + '/adata_reynolds_disease_per.h5ad')
adata_reynolds_disease_krt.write_h5ad(reynolds_dir + '/adata_reynolds_disease_krt.h5ad')
adata_reynolds_disease_lymphoid.write_h5ad(reynolds_dir + '/adata_reynolds_disease_lymphoid.h5ad')
adata_reynolds_disease_APC.write_h5ad(reynolds_dir + '/adata_reynolds_disease_APC.h5ad')

**RUN THIS CELL IF YOU ARE LOW ON RAM!**

In [ ]:
adata_reynolds_disease_fb = sc.read(reynolds_dir + '/adata_reynolds_disease_fb.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_disease_fb.h5ad?download=1')
adata_reynolds_disease_ve = sc.read(reynolds_dir + '/adata_reynolds_disease_ve.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_disease_ve.h5ad?download=1')
adata_reynolds_disease_per = sc.read(reynolds_dir + '/adata_reynolds_disease_per.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_disease_per.h5ad?download=1')
adata_reynolds_disease_krt = sc.read(reynolds_dir + '/adata_reynolds_disease_krt.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_disease_krt.h5ad?download=1')
adata_reynolds_disease_lymphoid = sc.read(reynolds_dir + '/adata_reynolds_disease_lymphoid.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_disease_lymphoid.h5ad?download=1')
adata_reynolds_disease_APC = sc.read(reynolds_dir + '/adata_reynolds_disease_APC.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_disease_APC.h5ad?download=1')

### Fibro

In [ ]:
assign_cats(adata_reynolds_disease_fb, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_disease_fb, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_disease_fb, color=[i for i in stress_genes if i in adata_reynolds_disease_fb.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_disease_fb, color=[i for i in hypoxia_genes if i in adata_reynolds_disease_fb.var_names], cmap=magma, use_raw=False)

### VE

In [ ]:
assign_cats(adata_reynolds_disease_ve, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_disease_ve, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_disease_ve, color=[i for i in stress_genes if i in adata_reynolds_disease_ve.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_disease_ve, color=[i for i in hypoxia_genes if i in adata_reynolds_disease_ve.var_names], cmap=magma, use_raw=False)

### PER

In [ ]:
assign_cats(adata_reynolds_disease_per, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_disease_per, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_disease_per, color=[i for i in stress_genes if i in adata_reynolds_disease_per.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_disease_per, color=[i for i in hypoxia_genes if i in adata_reynolds_disease_per.var_names], cmap=magma, use_raw=False)

### KRT

In [ ]:
assign_cats(adata_reynolds_disease_krt, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_disease_krt, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_disease_krt, color=[i for i in stress_genes if i in adata_reynolds_disease_krt.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_disease_krt, color=[i for i in hypoxia_genes if i in adata_reynolds_disease_krt.var_names], cmap=magma, use_raw=False)

### LYMPHOID

In [ ]:
assign_cats(adata_reynolds_disease_lymphoid, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_disease_lymphoid, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_disease_lymphoid, color=[i for i in stress_genes if i in adata_reynolds_disease_lymphoid.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_disease_lymphoid, color=[i for i in hypoxia_genes if i in adata_reynolds_disease_lymphoid.var_names], cmap=magma, use_raw=False)

### APC

In [ ]:
assign_cats(adata_reynolds_disease_APC, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_disease_APC, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_disease_APC, color=[i for i in stress_genes if i in adata_reynolds_disease_APC.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_disease_APC, color=[i for i in hypoxia_genes if i in adata_reynolds_disease_APC.var_names], cmap=magma, use_raw=False)

In [ ]:
if del_intermediates:
    del [adata_reynolds_disease_fb, adata_reynolds_disease_ve, adata_reynolds_disease_per, adata_reynolds_disease_krt, 
         adata_reynolds_disease_lymphoid, adata_reynolds_disease_APC]
    gc.collect()

# Analysis on all populations 

In [ ]:
for adata_name in ['adata_reynolds']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.normalize_per_cell(eval(adata_name))
    sc.pp.log1p(eval(adata_name))
    sc.pp.highly_variable_genes(eval(adata_name))
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds, color=['full_clustering'])

In [ ]:
sc.pl.umap(adata_reynolds, color=['BNIP3', 'BNIP3L'], cmap=magma, use_raw=False)

In [ ]:
adata_reynolds_fb = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['F1', 'F2', 'F3'])]
adata_reynolds_ve = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['VE1', 'VE2', 'VE3'])]
adata_reynolds_per = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Pericyte_1', 'Pericyte_2'])]
adata_reynolds_krt = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Differentiated_KC', 'Differentiated_KC*', 'Undifferentiated_KC'])]
adata_reynolds_lymphoid = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Tc', 'Tc17_Th17', 'Tc_IL13_IL22', 'Th', 'Treg', 'ILC1_3', 'ILC1_NK', 'ILC2'])]
adata_reynolds_APC = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Macro_1', 'Macro_2', 'Inf_mac', 'DC1', 'DC2', 'LC_1', 'LC_2', 'LC_3', 'Mono_mac', 'MigDC', 'MoDC_1', 'moDC_2', 'mo_DC3'])]

In [ ]:
samples, counts = np.unique(adata_reynolds_fb.obs['sample_id'], return_counts=True)
adata_reynolds_fb = adata_reynolds_fb[adata_reynolds_fb.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_ve.obs['sample_id'], return_counts=True)
adata_reynolds_ve = adata_reynolds_ve[adata_reynolds_ve.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_per.obs['sample_id'], return_counts=True)
adata_reynolds_per = adata_reynolds_per[adata_reynolds_per.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_krt.obs['sample_id'], return_counts=True)
adata_reynolds_krt = adata_reynolds_krt[adata_reynolds_krt.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_lymphoid.obs['sample_id'], return_counts=True)
adata_reynolds_lymphoid = adata_reynolds_lymphoid[adata_reynolds_lymphoid.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_APC.obs['sample_id'], return_counts=True)
adata_reynolds_APC = adata_reynolds_APC[adata_reynolds_APC.obs['sample_id'].isin(samples[counts > 10])]

In [ ]:
for adata_name in ['adata_reynolds_fb', 'adata_reynolds_ve', 'adata_reynolds_per', 'adata_reynolds_krt',
                   'adata_reynolds_lymphoid', 'adata_reynolds_APC']:
    sc.pp.filter_genes(eval(adata_name), min_counts=1)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)
    sc.tl.leiden(eval(adata_name), resolution=2.5, random_state=seed)

In [ ]:
adata_reynolds_fb.write_h5ad(reynolds_dir + '/adata_reynolds_fb.h5ad')
adata_reynolds_ve.write_h5ad(reynolds_dir + '/adata_reynolds_ve.h5ad')
adata_reynolds_per.write_h5ad(reynolds_dir + '/adata_reynolds_per.h5ad')
adata_reynolds_krt.write_h5ad(reynolds_dir + '/adata_reynolds_krt.h5ad')
adata_reynolds_lymphoid.write_h5ad(reynolds_dir + '/adata_reynolds_lymphoid.h5ad')
adata_reynolds_APC.write_h5ad(reynolds_dir + '/adata_reynolds_APC.h5ad')

**RUN THIS CELL IF YOU ARE LOW ON RAM!**

In [ ]:
adata_reynolds_fb = sc.read(reynolds_dir + '/adata_reynolds_fb.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_fb.h5ad?download=1')
adata_reynolds_ve = sc.read(reynolds_dir + '/adata_reynolds_ve.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_ve.h5ad?download=1')
adata_reynolds_per = sc.read(reynolds_dir + '/adata_reynolds_per.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_per.h5ad?download=1')
adata_reynolds_krt = sc.read(reynolds_dir + '/adata_reynolds_krt.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_krt.h5ad?download=1')
adata_reynolds_lymphoid = sc.read(reynolds_dir + '/adata_reynolds_lymphoid.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_lymphoid.h5ad?download=1')
adata_reynolds_APC = sc.read(reynolds_dir + '/adata_reynolds_APC.h5ad', backup_url='https://zenodo.org/record/4708700/files/adata_reynolds_APC.h5ad?download=1')

### Fibro

In [ ]:
assign_cats(adata_reynolds_fb, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_fb, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb, color=[i for i in stress_genes if i in adata_reynolds_fb.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb, color=[i for i in hypoxia_genes if i in adata_reynolds_fb.var_names], cmap=magma, use_raw=False)

### VE

In [ ]:
assign_cats(adata_reynolds_ve, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_ve, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_ve, color=[i for i in stress_genes if i in adata_reynolds_ve.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_ve, color=[i for i in hypoxia_genes if i in adata_reynolds_ve.var_names], cmap=magma, use_raw=False)

### PER

In [ ]:
assign_cats(adata_reynolds_per, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_per, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_per, color=[i for i in stress_genes if i in adata_reynolds_per.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_per, color=[i for i in hypoxia_genes if i in adata_reynolds_per.var_names], cmap=magma, use_raw=False)

### KRT

In [ ]:
assign_cats(adata_reynolds_krt, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_krt, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_krt, color=[i for i in stress_genes if i in adata_reynolds_krt.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_krt, color=[i for i in hypoxia_genes if i in adata_reynolds_krt.var_names], cmap=magma, use_raw=False)

### LYMPHOID

In [ ]:
assign_cats(adata_reynolds_lymphoid, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_lymphoid, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_lymphoid, color=[i for i in stress_genes if i in adata_reynolds_lymphoid.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_lymphoid, color=[i for i in hypoxia_genes if i in adata_reynolds_lymphoid.var_names], cmap=magma, use_raw=False)

### APC

In [ ]:
assign_cats(adata_reynolds_APC, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_APC, color=['full_clustering', 'leiden', 'hypoxia_stress', 
                                             'hypoxia_stress_Stress', 'hypoxia_stress_Hypoxia'], 
           cmap=magma, use_raw=False, ncols=3, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_APC, color=[i for i in stress_genes if i in adata_reynolds_APC.var_names], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_APC, color=[i for i in hypoxia_genes if i in adata_reynolds_APC.var_names], cmap=magma, use_raw=False)

In [ ]:
if del_intermediates:
    del [adata_reynolds_fb, adata_reynolds_ve, adata_reynolds_per, adata_reynolds_krt, adata_reynolds_lymphoid, adata_reynolds_APC]
    gc.collect()